In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Assuming 'data' is your dataset
PATH_TO_DATA = 'http://storage.googleapis.com/download.tensorflow.org/data/ecg.csv'
data = pd.read_csv(PATH_TO_DATA, header=None)

# Splitting data into train and test sets
X_train, X_test = train_test_split(data, test_size=0.2, random_state=42)

# Build the autoencoder model
input_dim = X_train.shape[1]  # Number of features

# Encoder
encoder = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(input_dim,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu')  # Latent representation
])

# Decoder
decoder = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(16,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(input_dim, activation='sigmoid')
])

# Combine Encoder and Decoder into an Autoencoder model
autoencoder_input = tf.keras.Input(shape=(input_dim,))
encoded = encoder(autoencoder_input)
decoded = decoder(encoded)
autoencoder = tf.keras.Model(autoencoder_input, decoded)

# Compile the model
optimizer = tf.keras.optimizers.Adam(lr=0.001)
autoencoder.compile(optimizer=optimizer, loss='mse')  # Mean Squared Error (MSE) as the loss function

# Training the autoencoder
autoencoder.fit(X_train, X_train, epochs=20, batch_size=32, shuffle=True, validation_data=(X_test, X_test))

# Using the trained encoder for anomaly detection
encoded_X_test = encoder.predict(X_test)

# Calculate reconstruction errors (MSE between original and reconstructed data)
reconstructions = autoencoder.predict(X_test)
mse = np.mean(np.power(X_test - reconstructions, 2), axis=1)

# Define a threshold for anomaly detection (e.g., based on the reconstruction error)
threshold = np.mean(mse) + 3 * np.std(mse)  # Example threshold calculation

# Identify anomalies based on the threshold
anomalies = np.where(mse > threshold)[0]

print("Indices of anomalies:", anomalies)


C:\Users\lenovo\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/20
125/125 [==============================] - 3s 8ms/step - loss: 0.8791 - val_loss: 0.7935
Epoch 2/20
125/125 [==============================] - 1s 4ms/step - loss: 0.7829 - val_loss: 0.7696
Epoch 3/20
125/125 [==============================] - 1s 4ms/step - loss: 0.7654 - val_loss: 0.7613
Epoch 4/20
125/125 [==============================] - 1s 4ms/step - loss: 0.7599 - val_loss: 0.7560
Epoch 5/20
125/125 [==============================] - 1s 4ms/step - loss: 0.7560 - val_loss: 0.7545
Epoch 6/20
125/125 [==============================] - 1s 4ms/step - loss: 0.7536 - val_loss: 0.7509
Epoch 7/20
125/125 [==============================] - 1s 4ms/step - loss: 0.7511 - val_loss: 0.7496
Epoch 8/20
125/125 [==============================] - 1s 4ms/step - loss: 0.7501 - val_loss: 0.7488
Epoch 9/20
125/125 [==============================] - 1s 4ms/step - loss: 0.7492 - val_loss: 0.7478
Epoch 10/20
125/125 [==============================] - 1s 4ms/step - loss: 0.7481 - val_loss: 0.7472